In [1]:
import numpy as np
import torch 
import yaml
import torch.nn as nn
import math 

from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import src.models as models
import src.dataset as ds
import src.train_pcdae as train_pcdae
import src.inference_pcdae as inference
import src.data_preparation as data_prep

In [2]:

config_file = "config_ebm_pcdae.yaml"
device = torch.device('cpu')

try:
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
except FileNotFoundError:
    print("Error: The file 'config.yaml' was not found.")
    exit()
except yaml.YAMLError as e:
    print(f"Error parsing YAML file: {e}")
    exit()
    


In [3]:
def inference_test(seed):
    
    ##! fix the seeds
    torch.manual_seed(seed)
    np.random.seed(seed)
    
    ###! load data and trained model
    data_dict = data_prep.prepare_data_model(seed, config['data']["data_path"], config['training']["ratio_test_val_train"])
    X_test_scaled, y_test_scaled = data_dict['test']
    scaler_X, scaler_Y = data_dict['scalers']
    batch_size = config['training']['batch_size']

    test_dataset = ds.LTPDataset(X_test_scaled, y_test_scaled)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    ###* define the PCDAE architecture
    x_dim = config['model']['x_dim']
    y_dim = config['model']['y_dim']
    hidden_dim = config['model']['hidden_dim']
        
    pcdae = models.PCDAE_EBM(x_dim=x_dim, y_dim=y_dim, hidden_dim=hidden_dim).to(device)
    pcdae.load_state_dict(torch.load(f"models_ebm_pcdae_saved/ebm_pcdae_cosine_inference_{seed}.pth"))
    
    ###! EBM sampling
    EBM_T = config['inference']['EBM_T']

    test_init_loss = 0
    test_T_loss = 0
    test_Tadapt_loss = 0
    
    constant_init_loss = 0
    constant_T_loss = 0
    constant_Tadapt_loss = 0

    for i, (x, y) in enumerate(test_dataloader):
        
        y_init = torch.randn_like(y)
        y_T = inference.inference_blind_adaptive_ODE(pcdae, x, y_init, eta=0.1, max_steps=EBM_T, eps_conv=1e-5, eps_clip=None)
        y_Tadapt = inference.inference_blind_adaptive_ODE(pcdae, x, y_init, eta=1.0, max_steps=EBM_T, eps_conv=1e-5, eps_clip=None, adaptative=True)
        
        constant_init_loss += inference.get_constraints_scalar(x, y_init, scaler_X, scaler_Y).pow(2).mean()
        constant_T_loss += inference.get_constraints_scalar(x, y_T, scaler_X, scaler_Y).pow(2).mean()
        constant_Tadapt_loss += inference.get_constraints_scalar(x, y_Tadapt, scaler_X, scaler_Y).pow(2).mean()
        
        
        loss_init = nn.MSELoss()(y_init, y)
        loss_T = nn.MSELoss()(y_T, y)
        loss_Tadapt = nn.MSELoss()(y_Tadapt, y)
        
        test_init_loss += loss_init.item()
        test_T_loss += loss_T.item()
        test_Tadapt_loss += loss_Tadapt.item()
        
        print("pro: ", (i+1)/len(test_dataloader))


    test_init_loss /= len(test_dataloader)
    test_T_loss /= len(test_dataloader)
    test_Tadapt_loss /= len(test_dataloader)
    
    constant_init_loss /= len(test_dataloader)
    constant_T_loss /= len(test_dataloader)
    constant_Tadapt_loss /= len(test_dataloader)
    
    return np.sqrt(test_init_loss), np.sqrt(test_T_loss), np.sqrt(test_Tadapt_loss), pcdae, constant_init_loss, constant_T_loss, constant_Tadapt_loss



In [4]:

# seed_vec = np.array([1, 41, 81, 121, 161, 201, 241, 281, 321, 361])
seed_vec = np.array([1])

rmse_init_vec = []
rmse_T_vec = []
rmse_Tadapt_vec = []

for i, seed in enumerate(seed_vec):
    
    val_init, val_T, val_Tadapt, pcdae, constraint_init, constraint_T, constraint_Tadapt = inference_test(seed)
    
    
    print(constraint_init.shape)
    
    
    rmse_init_vec.append(val_init)
    rmse_T_vec.append(val_T)
    rmse_Tadapt_vec.append(val_Tadapt)
    
    print("seed: ", seed)
    


steps:  214
pro:  0.1
steps:  134
pro:  0.2
steps:  209
pro:  0.3
steps:  132
pro:  0.4
steps:  201
pro:  0.5
steps:  140
pro:  0.6
steps:  159
pro:  0.7
steps:  132
pro:  0.8
steps:  162
pro:  0.9
steps:  185
pro:  1.0
torch.Size([])
seed:  1


In [5]:
rmse_init_arr = np.array(rmse_init_vec)
rmse_T_arr = np.array(rmse_T_vec)
rmse_Tadapt_arr = np.array(rmse_Tadapt_vec)

print(np.mean(rmse_init_arr))
print(np.mean(rmse_T_arr))
print(np.mean(rmse_Tadapt_arr))

1.4013526921070873
0.012999671364132681
0.012890143178316807


In [12]:
print(np.sqrt(constraint_init))
print(np.sqrt(constraint_T))
print(np.sqrt(constraint_Tadapt))

tensor(4.6361)
tensor(0.0139)
tensor(0.0143)


/var/folders/l6/sz7h_28d1vn668qy7qlhpvzw0000gn/T/ipykernel_29033/3854093083.py:1: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(np.sqrt(constraint_init))
/var/folders/l6/sz7h_28d1vn668qy7qlhpvzw0000gn/T/ipykernel_29033/3854093083.py:2: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(np.sqrt(constraint_T))
/var/folders/l6/sz7h_28d1vn668qy7qlhpvzw0000gn/T/ipykernel_29033/3854093083.py:3: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  print(np.sqrt(constraint_Tadapt))


In [7]:
# 0.01311752153355084

# 1.4013526921070873
# 0.01300977953589878
# 0.012890143178316807

In [8]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())


print(count_parameters(pcdae))

10480


In [9]:
# 1.4364558830118268
# 0.031021168598093285
# 0.01793517113448239